In [80]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil, csv, glob
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk
from nltk.tokenize import word_tokenize

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.8"))

from common import utils, vocabulary


In [81]:
# Load labeled data set
review_sentences = pd.read_csv('./data/Feature_sentences_Prod_Final_2018-07-09_Labeled_1K.csv', encoding='utf-8')
# Distribution by Sentiment Class
print(review_sentences.groupby('POS_SENTIMENT').size())

POS_SENTIMENT
0    213
1    826
dtype: int64


In [82]:
# Load inital extract for join/matching to obtain ratings
path =r'..' # use your path
allFiles = glob.glob(path + "/data/RR_Pared_All_Active_2018-07-09_*.csv")
all_reviews = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0)
    list_.append(df)
all_reviews = pd.concat(list_)

print(all_reviews.shape)


(369888, 8)


In [83]:
# Augment sentences (word features) with ratings data:

#print(review_sentences.head())
#print(all_reviews.head())
#rating = []
for index, row in review_sentences.iterrows():
    review_sentences.at[index, 'SENTENCE'] += ' '+str(all_reviews['OVERALL'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0])
    review_sentences.at[index, 'SENTENCE'] += ' '+str(all_reviews['ORGANIZATION'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0])
    review_sentences.at[index, 'SENTENCE'] += ' '+str(all_reviews['APPEARANCE'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0])
    review_sentences.at[index, 'SENTENCE'] += ' '+str(all_reviews['PRICE_VALUE'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0])
    review_sentences.at[index, 'SENTENCE'] += ' '+str(all_reviews['DURABILITY'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']].values[0])

    #rating.append(all_reviews['OVERALL'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']])
    #rating.append(all_reviews['ORGANIZATION'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']])
    #rating.append(all_reviews['APPEARANCE'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']])
    #rating.append(all_reviews['PRICE_VALUE'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']])
    #rating.append(all_reviews['DURABILITY'][all_reviews['TESTIMONIALID'] == row['TESTIMONIAL_ID']])

In [84]:
# Create training and test sets, keeping 80:20 ratio of sentiment class
dataset_class_0 = review_sentences[review_sentences['POS_SENTIMENT'] == 0]
dataset_class_1 = review_sentences[review_sentences['POS_SENTIMENT'] == 1]

#train = 170:660 = 830
#test = 43:166 = 209
#               == 1,039

train = dataset_class_0.iloc[0:170,]
train = train.append(dataset_class_1.iloc[0:660,])
print('train shape:', train.shape)

test = dataset_class_0.iloc[170:213,]
test = test.append(dataset_class_1.iloc[660:826,])
print('test shape:', test.shape)

train shape: (830, 4)
test shape: (209, 4)


In [85]:
train_x = train['SENTENCE']
train_y = train['POS_SENTIMENT']
test_x = test['SENTENCE']
test_y = test['POS_SENTIMENT']

In [86]:
# Tokenize sentences

train_x_tokens = []
test_x_tokens = []

for x in train_x:
    train_x_tokens.append(word_tokenize(x))
    
for x in test_x:
    test_x_tokens.append(word_tokenize(x))

In [87]:
# define vocabulary using w266 common vocab

train_text_all = [item for sublist in train_x_tokens for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=20000)
print("Vocabulary size: {:,}".format(vocab.size))
#print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []

for x in train_x_tokens:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))

Vocabulary size: 1,710


In [88]:
# NB - BoW Baseline

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

train_x_fdict = []
test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

train_x_vector = []
test_x_vector = []

num_features = vocab.size
print('creating id lists...')
for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# training Multinomial Naive Bayes for simple baseline model
print('training Multinomial Naive Bayes for simple baseline model...')
nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

creating id lists...
starting id lists to sparse bow conversion...
training Multinomial Naive Bayes for simple baseline model...
Accuracy on test set: 84.21%
             precision    recall  f1-score   support

          0       0.71      0.40      0.51        43
          1       0.86      0.96      0.91       166

avg / total       0.83      0.84      0.82       209



In [89]:
# Preprocessing for CNN model

from sklearn import preprocessing
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids

for x in train_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model --update your class lables below!!

terms_list = [[0],[1]]
lb = preprocessing.MultiLabelBinarizer()
lb.fit(terms_list)

train_labels = np.array(train_y.tolist())
test_labels = np.array(test_y.tolist())

train_lables_format = [train_labels[i:i+1] for i in range(0, len(train_labels), 1)]    
test_lables_format = [test_labels[i:i+1] for i in range(0, len(test_labels), 1)]

train_y_neural = lb.transform(train_lables_format)
test_y_neural = lb.transform(test_lables_format)

In [92]:
from Text_CNN import TextCNN

# CNN Model
# Leveraged Denny Britz template for CNN:
# http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/

# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_dev = np.array(test_x_neural)
y_dev = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_dev", np.shape(x_dev))
print("x_dev", np.shape(y_dev))

print("Vocabulary Size: {:d}".format(vocab.size))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))




Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f8b6d7bebe0>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f8b6d7bef60>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7f8b6d7bee80>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7f8b6d538908>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7f8b6d80ff28>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7f8b6d7bef28>
F=<absl.flags._flag.Flag object at 0x7f8b6d7bea20>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7f8b6d778550>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7f8b6d7befd0>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f8b6d7bed30>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7f8b6d7bec50>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7f8b6d7bee48>
NUM_FILTERS=<absl.flags._flag.Flag object at 0x7f8b6d7c3160>

x_train (830, 76)
y_train (830, 2)
x_dev (209, 76)
x_dev (209, 2)
Vocabulary Size: 1710
Train/Dev split: 830/209
INFO:tensorflow:Summary name embeddin

2018-08-10T17:54:46.042680: step 83, loss 0.430868, acc 0.859375
2018-08-10T17:54:46.197367: step 84, loss 0.409527, acc 0.875
2018-08-10T17:54:46.352900: step 85, loss 0.379597, acc 0.84375
2018-08-10T17:54:46.513703: step 86, loss 0.228107, acc 0.890625
2018-08-10T17:54:46.670874: step 87, loss 0.371715, acc 0.828125
2018-08-10T17:54:46.833380: step 88, loss 0.441709, acc 0.859375
2018-08-10T17:54:46.990967: step 89, loss 0.383134, acc 0.859375
2018-08-10T17:54:47.147132: step 90, loss 0.191923, acc 0.90625
2018-08-10T17:54:47.300448: step 91, loss 0.140194, acc 0.935484
2018-08-10T17:54:47.458620: step 92, loss 0.194591, acc 0.9375
2018-08-10T17:54:47.615166: step 93, loss 0.32854, acc 0.875
2018-08-10T17:54:47.777311: step 94, loss 0.267252, acc 0.9375
2018-08-10T17:54:47.932297: step 95, loss 0.409309, acc 0.875
2018-08-10T17:54:48.088874: step 96, loss 0.260496, acc 0.875
2018-08-10T17:54:48.245838: step 97, loss 0.302888, acc 0.859375
2018-08-10T17:54:48.401112: step 98, loss 0.

2018-08-10T17:55:05.239758: step 204, loss 0.0741588, acc 0.984375
2018-08-10T17:55:05.386124: step 205, loss 0.0621876, acc 0.96875
2018-08-10T17:55:05.537977: step 206, loss 0.129883, acc 0.953125
2018-08-10T17:55:05.693117: step 207, loss 0.0372406, acc 0.984375
2018-08-10T17:55:05.847780: step 208, loss 0.120228, acc 0.951613
2018-08-10T17:55:06.004748: step 209, loss 0.112203, acc 0.9375
2018-08-10T17:55:06.161991: step 210, loss 0.0570202, acc 0.984375
2018-08-10T17:55:06.319946: step 211, loss 0.125462, acc 0.9375
2018-08-10T17:55:06.481631: step 212, loss 0.0483198, acc 0.984375
2018-08-10T17:55:06.638719: step 213, loss 0.057819, acc 0.984375
2018-08-10T17:55:06.798293: step 214, loss 0.0325004, acc 0.984375
2018-08-10T17:55:06.955006: step 215, loss 0.104745, acc 0.953125
2018-08-10T17:55:07.112197: step 216, loss 0.0248563, acc 1
2018-08-10T17:55:07.268763: step 217, loss 0.0223161, acc 1
2018-08-10T17:55:07.424354: step 218, loss 0.0677887, acc 0.953125
2018-08-10T17:55:07.

2018-08-10T17:55:24.855311: step 328, loss 0.00290821, acc 1
2018-08-10T17:55:25.003984: step 329, loss 0.0207451, acc 1
2018-08-10T17:55:25.153744: step 330, loss 0.0576305, acc 0.96875
2018-08-10T17:55:25.304686: step 331, loss 0.0140056, acc 1
2018-08-10T17:55:25.452585: step 332, loss 0.00890186, acc 1
2018-08-10T17:55:25.601261: step 333, loss 0.136605, acc 0.953125
2018-08-10T17:55:25.748183: step 334, loss 0.0100075, acc 1
2018-08-10T17:55:25.897696: step 335, loss 0.047136, acc 0.984375
2018-08-10T17:55:26.047740: step 336, loss 0.0122091, acc 1
2018-08-10T17:55:26.194498: step 337, loss 0.152638, acc 0.96875
2018-08-10T17:55:26.337319: step 338, loss 0.0251671, acc 1
2018-08-10T17:55:26.489892: step 339, loss 0.00815219, acc 1
2018-08-10T17:55:26.646407: step 340, loss 0.0107403, acc 1
2018-08-10T17:55:26.803221: step 341, loss 0.0085333, acc 1
2018-08-10T17:55:26.961214: step 342, loss 0.0642681, acc 0.953125
2018-08-10T17:55:27.116581: step 343, loss 0.0149521, acc 1
2018-08

2018-08-10T17:55:44.744559: step 455, loss 0.0139315, acc 1
2018-08-10T17:55:44.895764: step 456, loss 0.00868863, acc 1
2018-08-10T17:55:45.045191: step 457, loss 0.00893528, acc 1
2018-08-10T17:55:45.192314: step 458, loss 0.0396543, acc 0.984375
2018-08-10T17:55:45.342885: step 459, loss 0.00856131, acc 1
2018-08-10T17:55:45.491999: step 460, loss 0.0153585, acc 1
2018-08-10T17:55:45.639632: step 461, loss 0.012088, acc 1
2018-08-10T17:55:45.787273: step 462, loss 0.00716194, acc 1
2018-08-10T17:55:45.935571: step 463, loss 0.00774224, acc 1
2018-08-10T17:55:46.084521: step 464, loss 0.00247498, acc 1
2018-08-10T17:55:46.231151: step 465, loss 0.0163791, acc 1
2018-08-10T17:55:46.378046: step 466, loss 0.00220665, acc 1
2018-08-10T17:55:46.527725: step 467, loss 0.0127984, acc 1
2018-08-10T17:55:46.671629: step 468, loss 0.00602419, acc 1
2018-08-10T17:55:46.821739: step 469, loss 0.00541734, acc 1
2018-08-10T17:55:46.969071: step 470, loss 0.00952735, acc 1
2018-08-10T17:55:47.1163

...

2018-08-10T17:59:46.300116: step 1987, loss 0.000952188, acc 1
2018-08-10T17:59:46.460108: step 1988, loss 7.6492e-05, acc 1
2018-08-10T17:59:46.616681: step 1989, loss 0.00105615, acc 1
2018-08-10T17:59:46.772265: step 1990, loss 0.00187364, acc 1
2018-08-10T17:59:46.932968: step 1991, loss 0.000255342, acc 1
2018-08-10T17:59:47.087717: step 1992, loss 0.000109057, acc 1
2018-08-10T17:59:47.245235: step 1993, loss 0.000243208, acc 1
2018-08-10T17:59:47.403378: step 1994, loss 5.89883e-05, acc 1
2018-08-10T17:59:47.560548: step 1995, loss 0.000237729, acc 1
2018-08-10T17:59:47.716395: step 1996, loss 6.3124e-05, acc 1
2018-08-10T17:59:47.875439: step 1997, loss 0.000375761, acc 1
2018-08-10T17:59:48.031257: step 1998, loss 9.50083e-05, acc 1
2018-08-10T17:59:48.181834: step 1999, loss 0.00051664, acc 1
2018-08-10T17:59:48.331193: step 2000, loss 0.000766452, acc 1

Evaluation:
2018-08-10T17:59:48.479843: step 2000, loss 0.792776, acc 0.856459

Saved model checkpoint to /home/mfrazzini/

2018-08-10T18:00:06.054419: step 2111, loss 0.000207576, acc 1
2018-08-10T18:00:06.212270: step 2112, loss 0.000421791, acc 1
2018-08-10T18:00:06.371969: step 2113, loss 8.28368e-05, acc 1
2018-08-10T18:00:06.530449: step 2114, loss 0.000221776, acc 1
2018-08-10T18:00:06.693351: step 2115, loss 0.00131546, acc 1
2018-08-10T18:00:06.848675: step 2116, loss 2.97146e-05, acc 1
2018-08-10T18:00:07.009874: step 2117, loss 0.000674346, acc 1
2018-08-10T18:00:07.168059: step 2118, loss 0.00129404, acc 1
2018-08-10T18:00:07.324162: step 2119, loss 0.000118801, acc 1
2018-08-10T18:00:07.481024: step 2120, loss 0.000220081, acc 1
2018-08-10T18:00:07.639389: step 2121, loss 0.000131333, acc 1
2018-08-10T18:00:07.798109: step 2122, loss 0.000194422, acc 1
2018-08-10T18:00:07.957019: step 2123, loss 0.000166198, acc 1
2018-08-10T18:00:08.116818: step 2124, loss 0.00010597, acc 1
2018-08-10T18:00:08.278120: step 2125, loss 5.85426e-05, acc 1
2018-08-10T18:00:08.442009: step 2126, loss 0.00787184, ac

2018-08-10T18:00:26.316840: step 2239, loss 0.000397379, acc 1
2018-08-10T18:00:26.480112: step 2240, loss 0.000237471, acc 1
2018-08-10T18:00:26.638243: step 2241, loss 0.000812172, acc 1
2018-08-10T18:00:26.795790: step 2242, loss 0.000184854, acc 1
2018-08-10T18:00:26.980658: step 2243, loss 4.15929e-05, acc 1
2018-08-10T18:00:27.137323: step 2244, loss 0.000727662, acc 1
2018-08-10T18:00:27.299063: step 2245, loss 0.000331248, acc 1
2018-08-10T18:00:27.463753: step 2246, loss 0.000172759, acc 1
2018-08-10T18:00:27.622805: step 2247, loss 8.85422e-05, acc 1
2018-08-10T18:00:27.782625: step 2248, loss 6.15504e-05, acc 1
2018-08-10T18:00:27.942484: step 2249, loss 8.57181e-05, acc 1
2018-08-10T18:00:28.106481: step 2250, loss 0.000229893, acc 1
2018-08-10T18:00:28.265746: step 2251, loss 0.00798341, acc 1
2018-08-10T18:00:28.432813: step 2252, loss 0.000140477, acc 1
2018-08-10T18:00:28.591415: step 2253, loss 4.65988e-05, acc 1
2018-08-10T18:00:28.752968: step 2254, loss 0.000107481,

2018-08-10T18:00:46.544980: step 2367, loss 0.00248014, acc 1
2018-08-10T18:00:46.701422: step 2368, loss 0.00721843, acc 1
2018-08-10T18:00:46.858791: step 2369, loss 0.00299472, acc 1
2018-08-10T18:00:47.021857: step 2370, loss 0.000654675, acc 1
2018-08-10T18:00:47.176932: step 2371, loss 0.000104019, acc 1
2018-08-10T18:00:47.337525: step 2372, loss 7.49709e-05, acc 1
2018-08-10T18:00:47.495189: step 2373, loss 0.000918958, acc 1
2018-08-10T18:00:47.652717: step 2374, loss 0.00332543, acc 1
2018-08-10T18:00:47.807230: step 2375, loss 8.35057e-05, acc 1
2018-08-10T18:00:47.966784: step 2376, loss 0.000642358, acc 1
2018-08-10T18:00:48.125720: step 2377, loss 0.000152724, acc 1
2018-08-10T18:00:48.280972: step 2378, loss 0.00229385, acc 1
2018-08-10T18:00:48.432201: step 2379, loss 0.000119651, acc 1
2018-08-10T18:00:48.590254: step 2380, loss 2.47296e-05, acc 1
2018-08-10T18:00:48.747853: step 2381, loss 0.000135672, acc 1
2018-08-10T18:00:48.907265: step 2382, loss 0.00187494, acc 

2018-08-10T18:01:06.813899: step 2495, loss 4.16855e-05, acc 1
2018-08-10T18:01:06.967102: step 2496, loss 0.00155708, acc 1
2018-08-10T18:01:07.123622: step 2497, loss 0.00114203, acc 1
2018-08-10T18:01:07.280303: step 2498, loss 0.000624285, acc 1
2018-08-10T18:01:07.436925: step 2499, loss 0.000164848, acc 1
2018-08-10T18:01:07.592750: step 2500, loss 0.000725553, acc 1

Evaluation:
2018-08-10T18:01:07.752824: step 2500, loss 0.877892, acc 0.856459

Saved model checkpoint to /home/mfrazzini/W266_project/deep_learning_sentiment_classification/runs/1533923672/checkpoints/model-2500

2018-08-10T18:01:07.974193: step 2501, loss 0.000113703, acc 1
2018-08-10T18:01:08.131749: step 2502, loss 0.0024418, acc 1
2018-08-10T18:01:08.288707: step 2503, loss 5.05958e-05, acc 1
2018-08-10T18:01:08.448794: step 2504, loss 0.0037111, acc 1
2018-08-10T18:01:08.612145: step 2505, loss 0.000133548, acc 1
2018-08-10T18:01:08.768785: step 2506, loss 0.00131054, acc 1
2018-08-10T18:01:08.929285: step 250